#Stock Indicators Python

In [1]:
import pandas as pd
import requests
import numpy as np
from lightweight_charts import Chart
from stock_indicators import indicators, Quote
from datetime import datetime, timedelta
import asyncio
import nest_asyncio

nest_asyncio.apply()

In [2]:
# import yfinance as yf
# df = yf.download('SPY', start='2010-01-01', multi_level_index=False)
# df.reset_index(inplace=True)
# df.to_csv('SPY.csv', index=False)
df = pd.read_csv('SPY.csv')
rawdf = df.copy()
df['Date'] = pd.to_datetime(df['Date'])
df.head()

,Date,Close,High,Low,Open,Volume
0,2010-01-04,85.279190,85.324337,83.909667,84.556805,118944600
1,2010-01-05,85.504951,85.542578,84.918014,85.226535,111579900
2,2010-01-06,85.565140,85.775835,85.354445,85.422166,116074400
3,2010-01-07,85.926353,86.031701,85.166341,85.407136,131091100
4,2010-01-08,86.212280,86.249907,85.527521,85.700590,126402800


In [3]:
quotes = [
    Quote(d, o, h, l, c, v)
    for d, o, h, l, c, v in zip(
        df['Date'],
        df['Open'],
        df['High'],
        df['Low'],
        df['Close'],
        df['Volume']
    )
]


In [4]:
# Moving Average Convergence / Divergence (MACD)
df['macd'] = [r.macd for r in indicators.get_macd(quotes, 12, 26, 9)]
df['signal'] = [r.signal for r in indicators.get_macd(quotes, 12, 26, 9)]
df['histogram'] = [r.histogram for r in indicators.get_macd(quotes, 12, 26, 9)]
df['bullishMACD'] = 0.0
df['bullishMACD'] = np.where(df['macd'] > df['signal'], 1.0, 0.0)
df['crossover_MACD'] = df['bullishMACD'].diff()

# Williams Alligator
df['jaw'] = [r.jaw for r in indicators.get_alligator(quotes, 13, 8, 8, 5, 5, 3)]
df['teeth'] = [r.teeth for r in indicators.get_alligator(quotes, 13, 8, 8, 5, 5, 3)]
df['lips'] = [r.lips for r in indicators.get_alligator(quotes, 13, 8, 8, 5, 5, 3)]

# Calculate EMA
df['EMA 12'] = [r.ema for r in indicators.get_ema(quotes, 12)]
df['EMA 20'] = [r.ema for r in indicators.get_ema(quotes, 20)]
df['EMA 25'] = [r.ema for r in indicators.get_ema(quotes, 25)]

# Calculate EMA12 crossover EMA25
df['bullishEMA'] = 0.0
df['bullishEMA'] = np.where(df['EMA 12'] > df['EMA 25'], 1.0, 0.0)
df['crossover_EMA12_EMA25'] = df['bullishEMA'].diff()

# Stochastic Momentum Index (SMI)
df['smi'] = [r.smi for r in indicators.get_smi(quotes, 14, 3, 3)]
df['smi_signal'] = [r.signal for r in indicators.get_smi(quotes, 14, 3, 3)]
df['bullishSMI'] = 0.0
df['bullishSMI'] = np.where(df['smi'] > df['smi_signal'], 1.0, 0.0)
df['crossover_SMI'] = df['bullishSMI'].diff()   

# Calculate Chandelier exit
from stock_indicators import ChandelierType 
df['chandelier_long_exit'] = [r.chandelier_exit for r in indicators.get_chandelier(quotes, 22, 3, ChandelierType.LONG)]
df['chandelier_short_exit'] = [r.chandelier_exit for r in indicators.get_chandelier(quotes, 22, 3, ChandelierType.SHORT)]
df['chandelier_exit'] = 0
df['chandelier_exit'] = np.where(df['chandelier_long_exit'] > df['chandelier_short_exit'], 1.0, 0.0)
df['crossover_chandelier_exit'] = df['chandelier_exit'].diff()  

# Calculate Bollinger Bands
df['BB_upper_band'] = [r.upper_band for r in indicators.get_bollinger_bands(quotes, 20, 2)]
df['BB_middle_band'] = [r.sma for r in indicators.get_bollinger_bands(quotes, 20, 2)]
df['BB_lower_band'] = [r.lower_band for r in indicators.get_bollinger_bands(quotes, 20, 2)]

# Arnaud Legoux Moving Average (ALMA)
df['ALMA_21'] = [r.alma for r in indicators.get_alma(quotes, 21, 0.85, 6)]
df['ALMA_50'] = [r.alma for r in indicators.get_alma(quotes, 50, 0.85, 6)]
df['alma_bullish'] = 0.0
df['alma_bullish'] = np.where(df['ALMA_21'] > df['ALMA_50'], 1.0, 0.0)
df['crossover_ALMA'] = df['alma_bullish'].diff()

# Ehlers Fisher Transform
df['fisher'] = [r.fisher for r in indicators.get_fisher_transform(quotes, 10)]
df['fisher_trigger'] = [r.trigger for r in indicators.get_fisher_transform(quotes, 10)]
df['bullishFisher'] = 0.0
df['bullishFisher'] = np.where(df['fisher'] > df['fisher_trigger'], 1.0, 0.0)
df['crossover_Fisher'] = df['bullishFisher'].diff()

# Choppiness Index
df['chopp'] = [r.chop for r in indicators.get_chop(quotes, 14)]

# Price Momentum Oscillator (PMO)
df['pmo'] = [r.pmo for r in indicators.get_pmo(quotes, 35, 20, 10)]
df['pmo_signal'] = [r.signal for r in indicators.get_pmo(quotes, 35, 20, 10)]
df['bullishPMO'] = 0.0
df['bullishPMO'] = np.where(df['pmo'] > df['pmo_signal'], 1.0, 0.0)
df['crossover_PMO'] = df['bullishPMO'].diff()

# Calculate RSI
df['rsi'] = [r.rsi for r in indicators.get_rsi(quotes, 14)]
df['rsima6'] = df['rsi'].rolling(6).mean()
df['rsima12'] = df['rsi'].rolling(12).mean()

# Calculate Mcginley dynamic
df['dynamic20'] = [r.dynamic for r in indicators.get_dynamic(quotes, 20)]


In [5]:
df = df.dropna().reset_index(drop=True)
df.head()


,Date,Close,High,Low,Open,Volume,macd,signal,histogram,bullishMACD,...,crossover_Fisher,chopp,pmo,pmo_signal,bullishPMO,crossover_PMO,rsi,rsima6,rsima12,dynamic20
0,2010-04-06,89.944763,90.103435,89.378075,89.476299,110384200,1.180809,1.120640,0.060169,1.0,...,0.0,54.872449,1.361479,1.150595,1.0,1.0,75.328624,71.542369,71.069816,87.378883
1,2010-04-07,89.430977,90.186561,89.015403,89.763436,184576300,1.157954,1.128103,0.029851,1.0,...,0.0,54.907568,1.378327,1.192000,1.0,0.0,68.518471,71.163574,70.937804,87.534727
2,2010-04-08,89.740730,89.891851,88.856698,89.121151,158704000,1.151561,1.132794,0.018766,1.0,...,-1.0,56.333453,1.404539,1.230644,1.0,0.0,70.263888,71.019888,70.753445,87.701140
3,2010-04-09,90.330147,90.367923,89.763458,89.929682,133006500,1.180448,1.142325,0.038123,1.0,...,1.0,52.180769,1.456288,1.271670,1.0,0.0,73.297664,72.071206,70.588897,87.895811
4,2010-04-12,90.473663,90.707898,90.337658,90.443439,110279000,1.201076,1.154075,0.047001,1.0,...,0.0,48.837679,1.500958,1.313359,1.0,0.0,73.993405,72.626019,70.925675,88.087175


In [6]:
# Moving Average Convergence / Divergence (MACD)
if __name__ == '__main__':
    
    chart = Chart(title="MACD Crossover", maximize=True, inner_height=0.8)
    chart.legend(visible=True, color_based_on_candle=True)

    # Set the main candlestick data for the chart.
    # The 'lightweight-charts' library expects a DataFrame with columns like 'Date', 'Open', 'High', 'Low', 'Close'.
    chart.set(df)

    # Create line series for EMAs
    macd_chart = chart.create_subchart(position='left', width=1.0, height=0.2, sync=True)
    macd_line = macd_chart.create_line('macd', color="#2ee30f", width=1, price_line=False, price_label=False)
    macd_line.set(df[['Date', 'macd']])

    signal_line = macd_chart.create_line('signal', color="#f36021", width=1, price_line=False, price_label=False)
    signal_line.set(df[['Date', 'signal']])

    histogram_bar = macd_chart.create_histogram('histogram', color="#1341e4", price_line=False, price_label=False)
    histogram_bar.set(df[['Date', 'histogram']])


    # Initialize a list to hold the markers
    markers = []

    # Iterate through the DataFrame to find crossover points
    for i in range(1, len(df)):

        MACD_diff = df.iloc[i]['crossover_MACD']
        
        current_time = df.iloc[i]['Date']

        # Check for buy signal (EMA 12 crosses above EMA 25)
        if MACD_diff == 1 :
            markers.append({
                'time': current_time,
                'position': 'below',
                'shape': 'arrow_up',
                'color': '#33de3d',
                'text': 'Buy'
            })
        
        # Check for sell signal (EMA 12 crosses below EMA 25)
        elif MACD_diff == -1 :
            markers.append({
                'time': current_time,
                'position': 'above',
                'shape': 'arrow_down',
                'color': '#f485fb',
                'text': 'Sell'
            })

    # Add all markers at once. It's more efficient than adding them individually in a loop.
    if markers:
        chart.marker_list(markers)
    
chart.show(block = True)

In [7]:
# Williams Alligator
if __name__ == '__main__':
    
    chart = Chart(title="Williams Alligator", maximize=True)
    chart.legend(visible=True, color_based_on_candle=True)

    # Set the main candlestick data for the chart.
    # The 'lightweight-charts' library expects a DataFrame with columns like 'Date', 'Open', 'High', 'Low', 'Close'.
    chart.set(df)

    # Create line series for EMAs
    jaw_line = chart.create_line('jaw', color="#1f77b4", width=1, price_line=False, price_label=False)
    jaw_line.set(df[['Date', 'jaw']])
    teeth_line = chart.create_line('teeth', color="#ff7f0e", width=1, price_line=False, price_label=False)
    teeth_line.set(df[['Date', 'teeth']])
    lips_line = chart.create_line('lips', color="#2ca02c", width=1, price_line=False, price_label=False)
    lips_line.set(df[['Date', 'lips']])

chart.show(block = True)

In [8]:
# Bollinger Bands
if __name__ == '__main__':

    chart = Chart(title="Bollinger Bands", maximize=True)
    chart.legend(visible=True, color_based_on_candle=True)

    # Set the main candlestick data for the chart.
    # The 'lightweight-charts' library expects a DataFrame with columns like 'Date', 'Open', 'High', 'Low', 'Close'.
    chart.set(df)

    # Create line series for EMAs
    upper_band_line = chart.create_line('BB_upper_band', color="#ff0000", width=1, price_line=False, price_label=False)
    upper_band_line.set(df[['Date', 'BB_upper_band']])
    middle_band_line = chart.create_line('BB_middle_band', color="#f3bd0b", width=1, price_line=False, price_label=False)
    middle_band_line.set(df[['Date', 'BB_middle_band']])
    lower_band_line = chart.create_line('BB_lower_band', color="#00ff00", width=1, price_line=False, price_label=False)
    lower_band_line.set(df[['Date', 'BB_lower_band']])

chart.show(block = True)

In [9]:
# Stochastic Momentum Index (SMI)
if __name__ == '__main__':
    
    chart = Chart(title="Stochastic Momentum Index (SMI)", maximize=True, inner_height=0.8)
    chart.legend(visible=True, color_based_on_candle=True)

    # Set the main candlestick data for the chart.
    # The 'lightweight-charts' library expects a DataFrame with columns like 'Date', 'Open', 'High', 'Low', 'Close'.
    chart.set(df)

    # Create line series for EMAs
    smi_chart = chart.create_subchart(position='left', width=1.0, height=0.2, sync=True)
    smi_line = smi_chart.create_line('smi', color="#2ee30f", width=1, price_line=False, price_label=False)
    smi_line.set(df[['Date', 'smi']])

    signal_line = smi_chart.create_line('smi_signal', color="#f36021", width=1, price_line=False, price_label=False)
    signal_line.set(df[['Date', 'smi_signal']])

    # Initialize a list to hold the markers
    markers = []

    # Iterate through the DataFrame to find crossover points
    for i in range(1, len(df)):

        SMI_diff = df.iloc[i]['crossover_SMI']
        
        current_time = df.iloc[i]['Date']

        # Check for buy signal (EMA 12 crosses above EMA 25)
        if SMI_diff == 1 :
            markers.append({
                'time': current_time,
                'position': 'below',
                'shape': 'arrow_up',
                'color': '#33de3d',
                'text': 'Buy'
            })
        
        # Check for sell signal (EMA 12 crosses below EMA 25)
        elif SMI_diff == -1 :
            markers.append({
                'time': current_time,
                'position': 'above',
                'shape': 'arrow_down',
                'color': '#f485fb',
                'text': 'Sell'
            })

    # Add all markers at once. It's more efficient than adding them individually in a loop.
    if markers:
        chart.marker_list(markers)
    
chart.show(block = True)

In [10]:
# Chandelier Exit

if __name__ == '__main__':
    
    chart = Chart(title="chandelier long_exit and short_exit Crossover", maximize=True)
    chart.legend(visible=True, color_based_on_candle=True)

    # Set the main candlestick data for the chart.
    # The 'lightweight-charts' library expects a DataFrame with columns like 'Date', 'Open', 'High', 'Low', 'Close'.
    chart.set(df)

    # Add Chandelier Exit lines
    chandelier_long_line = chart.create_line('Chandelier_Long_Exit', color='#8e44ad', width=1, price_line=False, price_label=False)
    chandelier_long_line.set(df[['Date', 'chandelier_long_exit']])
    # chandelier_short_line = chart.create_line('Chandelier_Short_Exit', color='#e74c3c', width=1, price_line=False, price_label=False)
    # chandelier_short_line.set(df[['Date', 'chandelier_short_exit']])

    
    # Initialize a list to hold the markers
    markers = []

    # Iterate through the DataFrame to find crossover points
    for i in range(1, len(df)):

        chan_diff = df.iloc[i]['crossover_chandelier_exit']
        
        current_time = df.iloc[i]['Date']

        # Check for buy signal (EMA 12 crosses above EMA 25)
        if chan_diff == 1 :
            markers.append({
                'time': current_time,
                'position': 'below',
                'shape': 'arrow_up',
                'color': '#33de3d',
                'text': 'Buy'
            })
        
        # Check for sell signal (EMA 12 crosses below EMA 25)
        elif chan_diff == -1 :
            markers.append({
                'time': current_time,
                'position': 'above',
                'shape': 'arrow_down',
                'color': '#f485fb',
                'text': 'Sell'
            })

    # Add all markers at once. It's more efficient than adding them individually in a loop.
    if markers:
        chart.marker_list(markers)
    
chart.show(block = True)


In [11]:
# Now, let's create the chart with lightweight-charts and add the EMAs and buy/sell markers.

# Assuming `df` is already a pandas DataFrame with 'Date', 'Open', 'High', 'Low', 'Close', 'EMA 12', and 'EMA 25' columns.
# It's good practice to convert the 'Date' column to the correct datetime format.
# df['Date'] = pd.to_datetime(df['Date'])

if __name__ == '__main__':
    
    chart = Chart(title="EMA12_EMA25 Crossover", maximize=True)
    chart.legend(visible=True, color_based_on_candle=True)

    # Set the main candlestick data for the chart.
    # The 'lightweight-charts' library expects a DataFrame with columns like 'Date', 'Open', 'High', 'Low', 'Close'.
    chart.set(df)

    # Create line series for EMAs
    ema12_line = chart.create_line('EMA 12', color='#ffeb3b', width=1, price_line=False, price_label=False)
    ema12_line.set(df[['Date', 'EMA 12']])

    ema25_line = chart.create_line('EMA 25', color='#26c6da', width=1, price_line=False, price_label=False)
    ema25_line.set(df[['Date', 'EMA 25']])
              
    # Initialize a list to hold the markers
    markers = []

    # Iterate through the DataFrame to find crossover points
    for i in range(1, len(df)):

        emadiff = df.iloc[i]['crossover_EMA12_EMA25']
        
        current_time = df.iloc[i]['Date']

        # Check for buy signal (EMA 12 crosses above EMA 25)
        if emadiff == 1 :
            markers.append({
                'time': current_time,
                'position': 'below',
                'shape': 'arrow_up',
                'color': '#33de3d',
                'text': 'Buy'
            })
        
        # Check for sell signal (EMA 12 crosses below EMA 25)
        elif emadiff == -1 :
            markers.append({
                'time': current_time,
                'position': 'above',
                'shape': 'arrow_down',
                'color': '#f485fb',
                'text': 'Sell'
            })

    # Add all markers at once. It's more efficient than adding them individually in a loop.
    if markers:
        chart.marker_list(markers)
    
chart.show(block = True)


In [12]:
# Arnaud Legoux Moving Average (ALMA)

if __name__ == '__main__':
    
    chart = Chart(title="Arnaud Legoux Moving Average (ALMA)", maximize=True)
    chart.legend(visible=True, color_based_on_candle=True)
    # chart.layout(background_color="white")

    # Set the main candlestick data for the chart.
    # The 'lightweight-charts' library expects a DataFrame with columns like 'Date', 'Open', 'High', 'Low', 'Close'.
    chart.set(df)

    # Create line series for EMAs
    alma21_line = chart.create_line('ALMA_21', color='#ffeb3b', width=1, price_line=False, price_label=False)
    alma21_line.set(df[['Date', 'ALMA_21']])

    alma50_line = chart.create_line('ALMA_50', color='#26c6da', width=1, price_line=False, price_label=False)
    alma50_line.set(df[['Date', 'ALMA_50']])
              
    # Initialize a list to hold the markers
    markers = []

    # Iterate through the DataFrame to find crossover points
    for i in range(1, len(df)):

        alma_diff = df.iloc[i]['crossover_ALMA']
        
        current_time = df.iloc[i]['Date']

        # Check for buy signal (EMA 12 crosses above EMA 25)
        if alma_diff == 1 :
            markers.append({
                'time': current_time,
                'position': 'below',
                'shape': 'arrow_up',
                'color': '#33de3d',
                'text': 'Buy'
            })
        
        # Check for sell signal (EMA 12 crosses below EMA 25)
        elif alma_diff == -1 :
            markers.append({
                'time': current_time,
                'position': 'above',
                'shape': 'arrow_down',
                'color': '#f485fb',
                'text': 'Sell'
            })

    # Add all markers at once. It's more efficient than adding them individually in a loop.
    if markers:
        chart.marker_list(markers)
    
chart.show(block = True)


In [13]:
# Ehlers Fisher Transform

if __name__ == '__main__':
    
    chart = Chart(title="Ehlers Fisher Transform", maximize=True, inner_height=0.8)
    chart.legend(visible=True, color_based_on_candle=True)
    # chart.layout(background_color="white")

    # Set the main candlestick data for the chart.
    # The 'lightweight-charts' library expects a DataFrame with columns like 'Date', 'Open', 'High', 'Low', 'Close'.
    chart.set(df)

    # Create line series for fisher transform
    fisher_chart = chart.create_subchart(position='left', width=1.0, height=0.2, sync=True)
    fisher_line = fisher_chart.create_line('fisher', color='#ffeb3b', width=1, price_line=False, price_label=False)
    fisher_line.set(df[['Date', 'fisher']])

    fisher_trigger_line = fisher_chart.create_line('fisher_trigger', color='#26c6da', width=1, price_line=False, price_label=False)
    fisher_trigger_line.set(df[['Date', 'fisher_trigger']])
              
    # Initialize a list to hold the markers
    markers = []

    # Iterate through the DataFrame to find crossover points
    for i in range(1, len(df)):

        fisher_diff = df.iloc[i]['crossover_Fisher']
        
        current_time = df.iloc[i]['Date']

        # Check for buy signal (EMA 12 crosses above EMA 25)
        if fisher_diff == 1 :
            markers.append({
                'time': current_time,
                'position': 'below',
                'shape': 'arrow_up',
                'color': '#33de3d',
                'text': 'Buy'
            })
        
        # Check for sell signal (EMA 12 crosses below EMA 25)
        elif fisher_diff == -1 :
            markers.append({
                'time': current_time,
                'position': 'above',
                'shape': 'arrow_down',
                'color': '#f485fb',
                'text': 'Sell'
            })

    # Add all markers at once. It's more efficient than adding them individually in a loop.
    if markers:
        chart.marker_list(markers)
    
chart.show(block = True)


In [14]:
# Choppiness Index

if __name__ == '__main__':

    chart = Chart(title="Choppiness Index", maximize=True, inner_height=0.8)
    chart.legend(visible=True, color_based_on_candle=True)
    # chart.layout(background_color="white")

    # Set the main candlestick data for the chart.
    # The 'lightweight-charts' library expects a DataFrame with columns like 'Date', 'Open', 'High', 'Low', 'Close'.
    chart.set(df)

    # Create line series for choppiness index
    chop_chart = chart.create_subchart(position='left', width=1.0, height=0.2, sync=True)
    chop_line = chop_chart.create_line('chopp', color='#ffeb3b', width=1, price_line=False, price_label=False)
    chop_line.set(df[['Date', 'chopp']])
    # chop60_line = chop_chart.horizontal_line(price=60, color='#26c6da', width=1, style='dashed', text='60')
    # chop40_line = chop_chart.horizontal_line(price=40, color='#ff5733', width=1, style='dashed', text='40')

chart.show(block = True)


In [15]:
# Price Momentum Oscillator (PMO)

if __name__ == '__main__':

    chart = Chart(title="Price Momentum Oscillator (PMO)", maximize=True, inner_height=0.8)
    chart.legend(visible=True, color_based_on_candle=True)
    # chart.layout(background_color="white")

    # Set the main candlestick data for the chart.
    # The 'lightweight-charts' library expects a DataFrame with columns like 'Date', 'Open', 'High', 'Low', 'Close'.
    chart.set(df)

    # Create line series for PMO
    pmo_chart = chart.create_subchart(position='left', width=1.0, height=0.2, sync=True)
    pmo_line = pmo_chart.create_line('pmo', color='#ffeb3b', width=1, price_line=False, price_label=False)
    pmo_line.set(df[['Date', 'pmo']])

    pmo_signal_line = pmo_chart.create_line('pmo_signal', color='#26c6da', width=1, price_line=False, price_label=False)
    pmo_signal_line.set(df[['Date', 'pmo_signal']])
              
    # Initialize a list to hold the markers
    markers = []

    # Iterate through the DataFrame to find crossover points
    for i in range(1, len(df)):

        pmo_diff = df.iloc[i]['crossover_PMO']
        
        current_time = df.iloc[i]['Date']

        # Check for buy signal (EMA 12 crosses above EMA 25)
        if pmo_diff == 1 :
            markers.append({
                'time': current_time,
                'position': 'below',
                'shape': 'arrow_up',
                'color': '#33de3d',
                'text': 'Buy'
            })
        
        # Check for sell signal (EMA 12 crosses below EMA 25)
        elif pmo_diff == -1 :
            markers.append({
                'time': current_time,
                'position': 'above',
                'shape': 'arrow_down',
                'color': '#f485fb',
                'text': 'Sell'
            })

    # Add all markers at once. It's more efficient than adding them individually in a loop.
    if markers:
        chart.marker_list(markers)
    
chart.show(block = True)
